<a href="https://colab.research.google.com/github/LarrySnyder/stockpyl/blob/master/notebooks/Stockpyl_Tutorial_%C2%A72_Single_Echelon_Inventory_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stockpyl Tutorial
=================

(This notebook is a companion to Snyder, L. V., "[Stockpyl: A Python Package for Inventory Optimization and Simulation](https://pubsonline.informs.org/doi/10.1287/educ.2023.0256)," in: Bish, E. K. and H. Balasubramanian, _INFORMS TutORials in Operations Research_, 156–197, 2023.)



# Section 2: Single-Echelon Inventory Optimization

Stockpyl contains code to solve the following types of single-echelon inventory optimization problems:

* The economic order quantity (EOQ) problem (Section 2.1)
* The newsvendor problem (Section 2.2)
* The $(r,Q)$ and $(s,S)$ optimization problems (Sections 2.3 and 2.4)
* The Wagner--Whitin problem (Section 2.6)
* Finite-horizon stochastic problems with or without fixed costs (Section 2.7)
* Plus a number of single-echelon problems with supply uncertainty (see Section 3)


First, install the package:

In [ ]:
!pip install stockpyl

## 2.1 The EOQ Problem and Variants

The `eoq` module contains code for solving the economic order quantity (EOQ) problem and some of its variants. The EOQ problem is arguably the oldest and simplest inventory optimization problem, dating back to Harris (1913). The problem assumes that the demand is continuous, deterministic, and constant, and that there is a fixed cost to place an order and a holding cost to store inventory. The goal of the problem is to determine the optimal order quantity to use for each order. Many variants of the EOQ have been proposed in the literature, and Stockpyl implements some of the more common ones.

The main function in the module is the [`economic_order_quantity()`](https://stockpyl.readthedocs.io/en/latest/api/seio/eoq.html#stockpyl.eoq.economic_order_quantity) function, which implements the basic EOQ model (Harris 1913; _FoSCT_ §3.2).

The function has the signature
```python
economic_order_quantity(fixed_cost, holding_cost, demand_rate, order_quantity=None)
```
and returns two parameters: `order_quantity` (the optimal order quantity) and `cost` (the corresponding optimal cost).

**Example:** If the fixed cost is 8, the holding cost per item per year is 0.225, and the demand rate is 1300 items per year (_FoSCT_ Example 3.1), then the optimal order quantity is 304.05 and the optimal average cost per year is 68.41:

In [ ]:
from stockpyl.eoq import economic_order_quantity
economic_order_quantity(8, 0.225, 1300)

Alternately, you can pass an order quantity $Q$ to `economic_order_quantity()` in the optional parameter `order_quantity`, and the function will return the cost of that order quantity (and the order quantity itself). (Many functions in Stockpyl have such an option.)

In [ ]:
economic_order_quantity(8, 0.225, 1300, order_quantity=350)

---
**Exercise:** Solve the EOQ problem with a fixed cost of 150, a holding cost of 2 per item per year, and a demand rate of 200 items per year.

**Exercise:** What is the average cost per year if we use an order quantity of 50?

---
The `eoq` module contains functions to solve a few variants of the EOQ. In particular:

In [ ]:
# EOQ with backorders (EOQB).
from stockpyl.eoq import economic_order_quantity_with_backorders
economic_order_quantity_with_backorders(fixed_cost=8, holding_cost=0.225,
                                        stockout_cost=5, demand_rate=1300)

In [ ]:
# Economic production quantity (EPQ).
from stockpyl.eoq import economic_production_quantity
economic_production_quantity(fixed_cost=8, holding_cost=0.225, demand_rate=1300,
                             production_rate=1700)

It also solves the joint replenishment problem (JRP) using Silver’s (1976) heuristic. The [`joint_replenishment_problem_silver_heuristic()`](https://stockpyl.readthedocs.io/en/latest/api/seio/eoq.html#stockpyl.eoq.joint_replenishment_problem_silver_heuristic) function has the following signature:

```python
joint_replenishment_problem_silver_heuristic(shared_fixed_cost, individual_fixed_costs, holding_costs, demand_rates)
```

It returns four parameters: `order_quantities` (the order quantities found by the heuristic), `base_cycle_time` (the intervals between consecutive orders), `order_multiples` (product `n` is included in every `order_multiples[n]` orders, and `cost` (cost per unit time).


**Example:** Suppose there are 3 products. Each order (regardless of its contents) incurs a shared fixed cost of 600. Each order that contains products 1, 2, or 3 incurs an additional individual fixed cost of 120, 840, and 300, respectively. The holding costs for products 1, 2, and 3 are 160, 20, and 50 per item per week, respectively. The demand rate is 1 item per week for each product.

Then the optimal order quantities are $Q^* = [3.1, 9.3, 3.1]$; we should order every 3.1 weeks; we order products 1 and 3 in every order, and product 2 every 3 orders; and we incur an average cost of 837.9 per week.

In [ ]:
from stockpyl.eoq import joint_replenishment_problem_silver_heuristic
shared_fixed_cost = 600
individual_fixed_costs = [120, 840, 300]
holding_costs = [160, 20, 50]
demand_rates = [1, 1, 1]
joint_replenishment_problem_silver_heuristic(shared_fixed_cost, individual_fixed_costs,
                                             holding_costs, demand_rates)


## 2.2 The Newsvendor Problem

The newsvendor problem (Arrow, Harris, and Marschak (1951); _FoSCT_ §4.3.2) is perhaps the simplest stochastic inventory problem. The `newsvendor` module contains code for solving several flavors of this problem.


> **Remark:** The functions in the `newsvendor` module solve either the single-period newsvendor problem, or its infinite-horizon analogue. In the infinite-horizon version, unsold items may be held from one period to the next in the form of inventory (incurring a holding cost), and unmet demands may be held from one period to the next in the form of backorders (incurring a stockout cost). We use the term "newsvendor" to refer to both the single-period and infinite-horizon problems, since they are mathematically equivalent. However, some authors prefer not to use the term "newsvendor" to describe the infinite-horizon problem. That problem may alternatively be called the _base-stock optimization problem_, since the firm is following a base-stock policy and the objective is to find the optimal base-stock level.

The [`newsvendor_normal()`](https://stockpyl.readthedocs.io/en/latest/api/seio/newsvendor.html#stockpyl.newsvendor.newsvendor_normal) function solves the newsvendor problem with normally distributed demands (_FoSCT_ §4.3.2.5). This version of the problem assumes the demand per period is normally distributed.

The function has the signature
```python
newsvendor_normal(holding_cost, stockout_cost, demand_mean, demand_sd,
    lead_time=0, base_stock_level=None)
```
and returns two parameters: `base_stock_level` (the optimal base-stock level) and `cost` (the corresponding optimal expected cost).


**Example:** Suppose the holding cost is 0.18 per item, the stockout cost is 0.70 per item, and the demand is distributed as $N(50,8^2)$ (_FoSCT_ Example 4.3). Then the optimal base-stock level is 56.6 and the optimal expected cost is 2.00.

In [ ]:
from stockpyl.newsvendor import newsvendor_normal
newsvendor_normal(0.18, 0.70, 50, 8)

Alternatively, you can pass a base-stock level $S$ to `newsvendor_normal()` in the optional parameter `base_stock_level`, and the function will return the expected cost of that base-stock level (and the base-stock level itself).

In [ ]:
newsvendor_normal(0.18, 0.70, 50, 8, base_stock_level=60)

Non-zero lead times can be handled by providing the `newsvendor_normal()` function with a lead time $L$ in the `lead_time` parameter:


In [ ]:
newsvendor_normal(0.18, 0.70, 50, 8, lead_time=3)

---
**Exercise:** Solve the newsvendor problem with a holding cost of 3 per item, a stockout cost of 16 per item, and a demand mean and standard deviation of 100 and 22.

**Exercise:** Suppose in addition that there is a lead time of 4 periods.

---
The [`newsvendor_poisson()`](https://stockpyl.readthedocs.io/en/latest/api/seio/newsvendor.html#stockpyl.newsvendor.newsvendor_poisson) function solves the problem for Poisson demands. It has a similar signature as `newsvendor_normal()` except that it does not have a parameter for the standard deviation.


In [ ]:
from stockpyl.newsvendor import newsvendor_poisson
newsvendor_poisson(0.18, 0.70, 50)

---
Stockpyl provides two functions that allow you to solve newsvendor problems with generic distributions (i.e., not normal or Poisson). The [`newsvendor_continuous()`](https://stockpyl.readthedocs.io/en/latest/api/seio/newsvendor.html#stockpyl.newsvendor.newsvendor_continuous) function is for continuous distributions. It allows you to specify the demand pdf in one of two ways:
* as a [`scipy.stats.rv_continuous`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html) object, or
* as a function that takes a single argument and returns the pdf of that argument. (🤦 This has not been implemented yet.)

The signature of the function is:
```python
newsvendor_continuous(holding_cost, stockout_cost, demand_distrib=None,
    demand_pdf=None, base_stock_level=None)
```
(Exactly one of `demand_distrib` and `demand_pdf` must be provided.)

**Example:** Suppose the holding cost is 1 per item, the stockout cost is 0.1765 per item, and the demand is lognormal with paramters $\mu=6$ and $\sigma=0.3$ (_FoSCT_ Problem 4.9(b)). Then the optimal base-stock level is 295.6 and the optimal expected cost is 29.4.

(_Note_: To create a lognormal random variable with parameters $\mu$ and $\mu$ using `scipy.stats.lognorm()`, set the first parameter to $\sigma$, the second to 0, and the third to $e^\mu$.)

In [ ]:
from stockpyl.newsvendor import newsvendor_continuous
import numpy as np
from scipy.stats import lognorm
demand_distrib = lognorm(0.3, 0, np.exp(6))
newsvendor_continuous(1, 0.1765, demand_distrib)

For discrete distributions, the [`newsvendor_discrete()`](https://stockpyl.readthedocs.io/en/latest/api/seio/newsvendor.html#stockpyl.newsvendor.newsvendor_discrete) function allows you to specify the demand pmf either:
* as a [scipy.stats.rv_discrete](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html) object, or
* as a Python dictionary in which the keys are the possible demand values and the values are their probabilities.

The signature is:
```python
newsvendor_discrete(holding_cost, stockout_cost, demand_distrib=None,
    demand_pmf=None, base_stock_level=None)
```
(Exactly one of `demand_distrib` and `demand_pmf` must be provided.)

**Example:** In _FoSCT_ Example 4.7, the demand has a Poisson distribution with mean 6, the holding cost is 1, and the stockout cost is 4. Let's first solve this problem by passing `newsvendor_discrete()` an `rv_discrete` object that specifies the demand distribution:

In [ ]:
from stockpyl.newsvendor import newsvendor_discrete
from scipy.stats import poisson
demand_distrib = poisson(6)
newsvendor_discrete(1, 4, demand_distrib=demand_distrib)


Instead of building an `rv_discrete` object, we can instead pass a dictionary that specifies the pmf explicitly:

In [ ]:
from scipy.stats import poisson
# Build the pmf dictionary.
d = range(0, 41)
f = [poisson.pmf(d_val, 6) for d_val in d]
demand_pmf = dict(zip(d, f))
# Now we have a dict in which demand_pmf[0] = 0.0024787521766663585,
# demand_pmf[1] = 0.014872513059998144, and so on.
# Solve the problem.
newsvendor_discrete(1, 4, demand_pmf=demand_pmf)

Note that we get the same optimal solution and cost either way.

---
**Exercise:** Suppose the holding cost is 10, the stockout cost is 100, and the demand has a geomtric distribution with parameter $p=0.1$. Solve the resulting newsvendor problem.

**Exercise:** Suppose instead that the demand equals $0, ..., 5$ with probabilities $0.1, 0.2, 0.2, 0.25, 0.15, 0.1$, respectively. Solve the resulting problem.


---
The functions discussed so far use the "implicit" form of the newsvendor problem, in which we provide holding and stockout (overage and underage) costs. Another version of the problem, the "explicit" version, optimizes based on the revenue $r$ per item sold, the cost $c$ per item purchased, and the salvage value $v$ per item left over at the end of the period.  Note that, if we let
	\begin{align}
		h' & = c - v \\
		p' & = r - c,
	\end{align}
then the explicit formulation under $r$, $c$, and $v$ is equivalent to the implicit formulation under $h'$ and $p'$.

Stockpyl provides two functions to solve the explicit form of the newsvendor problem: [`newsvendor_normal_explicit()`](https://stockpyl.readthedocs.io/en/latest/api/seio/newsvendor.html#stockpyl.newsvendor.newsvendor_normal_explicit), which has the signature
```python
newsvendor_normal_explicit(revenue, purchase_cost, salvage_value,
    demand_mean, demand_sd, holding_cost=0, stockout_cost=0, lead_time=0,
    base_stock_level=None)
```
and [`newsvendor_poisson_explicit()`](https://stockpyl.readthedocs.io/en/latest/api/seio/newsvendor.html#stockpyl.newsvendor.newsvendor_poisson_explicit), which has the signature
```python
newsvendor_poisson_explicit(revenue, purchase_cost, salvage_value,
    demand_mean, holding_cost=0, stockout_cost=0, lead_time=0,
    base_stock_level=None)
```
(The optional `holding_cost` and `stockout_cost` parameters allow you to specify additional holding and stockout costs, in addition to the costs implied by the revenue, purchase cost, and salvage value.)

**Example:** Suppose we have a revenue of $r=1$, a purchase cost of $c=0.3$, a salvage value of $v=0.12$, and demand distributed as $N(50,8^2)$. Then the optimal base-stock level is 56.6 and the optimal expected cost is 33.0:

In [ ]:
from stockpyl.newsvendor import newsvendor_normal_explicit
newsvendor_normal_explicit(1, 0.3, 0.12, 50, 8)

If, instead, the demand is distributed as $\text{Pois}(50)$, the optimal order quantity and cost change only a little:

In [ ]:
from stockpyl.newsvendor import newsvendor_poisson_explicit
newsvendor_poisson_explicit(1, 0.3, 0.12, 50)

---
**Exercise:** Solve the newsvendor problem with revenue $r=500$, purchase cost $c=200$, salvage value $v=80$, and demand distributed as $N(12, 2^2)$.

In [ ]:
newsvendor_normal_explicit(500, 200, 80, 12, 2)


## 2.3 The $(r,Q)$ Optimization Problem

The `rq` module contains functions for evaluating and optimizing $(r,Q)$ policies. The functions in this module assume a continuous-review system; different functions make different assumptions about the demand distribution. The module offers functions for both exact and approximate optimization methods to find $r$ and $Q$.

Four functions implement heuristic (approximate) methods for finding $r$ and $Q$. These are discussed in _FoSCT_ §5.3.

The [`r_q_eil_approximation()`](https://stockpyl.readthedocs.io/en/latest/api/seio/rq.html#stockpyl.rq.r_q_eil_approximation) function implements the "expected-inventory-level" (EIL) approximation (Hadley and Whitin 1963, Whitin 1953), which assumes the demand per unit time is distributed as $N(\lambda,\tau^2)$. This approach iteratively solves a pair of equations for $Q$ and $r$ to derive an approximate solution. The function has the signature
```python
r_q_eil_approximation(holding_cost, stockout_cost, fixed_cost, demand_mean,
    demand_sd, lead_time, tol=1e-6)
```
and returns three parameters: `reorder_point` (the approximate reorder point), `order_quantity` (the approximate order quantity), and `cost` (the corresponding expected cost). The optional parameter `tol` allows you to specify the tolerance for the iterative method.



**Example:** Suppose the holding cost is 0.225 item per year, the stockout cost is 7.5 per item per year, the fixed cost is 8, the annual demand is distributed as $N(1300,150^2)$, and the lead time is 1 month (1/12 year) (_FoSCT_ Example 5.2). The EIL approximation gives a reorder point of $r=214.0$, an order quantity of $318.6$, for a corresponding expected cost of 95.5 per year:

In [ ]:
from stockpyl.rq import r_q_eil_approximation
r_q_eil_approximation(0.225, 7.5, 8, 1300, 150, 1/12)

The other three functions for $(r,Q)$ approximations work similarly:
* [`r_q_eoqb_approximation()`](https://stockpyl.readthedocs.io/en/latest/api/seio/rq.html#stockpyl.rq.r_q_eoqb_approximation) implements the "economic order quantity with backorders" (EOQB) approximation
* [`r_q_eoqss_approximation()`](https://stockpyl.readthedocs.io/en/latest/api/seio/rq.html#stockpyl.rq.r_q_eoqss_approximation) implements the "EOQ plus safety stock" (EOQSS) approximation
* [`r_q_loss_function_approximation()`](https://stockpyl.readthedocs.io/en/latest/api/seio/rq.html#stockpyl.rq.r_q_loss_function_approximation) implements the "loss function" approximation (Hadley and Whitin 1963).

These functions all assume that the demand is normally distributed. They do not return the expected cost of the solution found; to calculate the expected cost, use the [`r_q_cost()`](https://stockpyl.readthedocs.io/en/latest/api/seio/rq.html#stockpyl.rq.r_q_cost) function, which calculates the _exact_ expected cost of given values of $r$ and $Q$, assuming the demand is normally distributed.






**Example:** Solving the same problem as above using the EOQ+SS approximation, we get $r=190.3$ and $Q=304.1$, with an expected cost of 87.1 (a bit better than the solution returned by the EIL approximation).

In [ ]:
from stockpyl.rq import r_q_eoqss_approximation
r, Q = r_q_eoqss_approximation(0.225, 7.5, 8, 1300, 150, 1/12)
r, Q

In [ ]:
from stockpyl.rq import r_q_cost
r_q_cost(r, Q, 0.225, 7.5, 8, 1300, 150, 1/12)

---
**Exercise:** Suppose the holding cost is 2 per week, the stockout cost is 16, the fixed cost is 75, the weekly demand is distributed as $N(40, 6^2)$, and the lead time is 2 weeks. Using all four approximations, find $r$, $Q$, and the associated cost.


## 2.4 The $(s,S)$ Optimization Problem

The `ss` module contains functions for evaluating and optimizing $(s,S)$ policies in a periodic-review system. The notation is mostly the same as in the `rq` module. The [`s_s_cost_discrete()`](https://stockpyl.readthedocs.io/en/latest/api/seio/ss.html#stockpyl.ss.s_s_cost_discrete) function calculates the exact expected cost of an $(s,S)$ policy with given parameters $s$ and $S$ under a discrete distribution using the method by (Zheng and Federgruen 1991, 1992). Its signature is
```python
s_s_cost_discrete(reorder_point, order_up_to_level, holding_cost,
    stockout_cost, fixed_cost, use_poisson, demand_mean=None, demand_hi=None,
    demand_pmf=None)
```
The distribution can be Poisson (in which case, set `use_poisson=True` and provide the mean of the distribution in the argument `demand_mean`) or an arbitrary discrete distribution. In the latter case, the distribution is assumed to be on the integers $0, \ldots, $`demand_hi`; set `use_poisson=False` and provide the pmf of the distribution as a list of probabilities in `demand_pmf`.




**Example:** Suppose we use $(s,S)=(4,10)$ for a problem with holding cost 1, stockout cost 4, fixed cost 5, and $\text{Pois}(6)$ demand (_FoSCT_ Example 4.7). Then the expected cost is 8.03:

In [ ]:
from stockpyl.ss import s_s_cost_discrete
h = 1
p = 4
K = 5
mu = 6
s_s_cost_discrete(4, 10, h, p, K, use_poisson=True, demand_mean=mu)

**Example:** Suppose instead that the demand has a geometric distribution with the same mean, 6, as in the previous example, truncatnig the support at 20. Then the expected cost is 9.89:

In [ ]:
from scipy.stats import geom
demand_prob = 1/6
demand_hi = 20
demand_pmf = [geom.pmf(x, demand_prob) for x in range(demand_hi + 1)]
s_s_cost_discrete(4, 10, h, p, K, use_poisson=False, demand_hi=demand_hi,
    demand_pmf=demand_pmf)

The [`s_s_discrete_exact`](https://stockpyl.readthedocs.io/en/latest/api/seio/ss.html#stockpyl.ss.s_s_discrete_exact) function finds the exact optimal $s$ and $S$ for a policy under discrete (Poisson or arbitrary) demands using the algorithm by Zheng and Federgruen (1991), i.e., it optimizes the cost calculated by `s_s_cost_discrete()`. Its signature is similar, except that `reorder_point` and `order_up_to_level` are outputs rather than inputs.

**Example:** Optimizing the instance from _FoSCT_ Example 4.7, we find that $(s^*,S^*) = (4, 10)$, with expected cost 8.03:


In [ ]:
from stockpyl.ss import s_s_discrete_exact
s_s_discrete_exact(h, p, K, True, 6)

Finally, the [`s_s_power_approximation()`](https://stockpyl.readthedocs.io/en/latest/api/seio/ss.html#stockpyl.ss.s_s_power_approximation) function finds heuristic values for $s$ and $S$ using the "power approximation" by Erhardt and Mosier (1984), which assumes the demands are normally distributed.

In [ ]:
from stockpyl.ss import s_s_power_approximation
s_s_power_approximation(h, p, K, True, 6)

---
**Exercise:** Use both the exact algorithm and the power approximation to solve the $(s,S)$ optimization problem with a holding cost of 12, a stockout cost of 76, a fixed cost of 225, and $\text{Pois}(65)$ demand. Compare the expected costs of the two solutions.

## 2.5 Specification of Time-Based Quantities
<a id='sec_2_5'></a>

Before moving on, we take a quick detour to discuss how time-based input and output parameters (i.e., parameters with a subscript $t$) are represented in Stockpyl. These are relevant in the section below and in subsequent notebooks.

Let $T$ be the number of time periods. In most cases, if a parameter may differ from one period to the next, Stockpyl allows you to specify it in one of three ways:
* As a singleton, in which case the parameter is assumed to equal that value in every time period.
* As a list with $T$ elements, in which case the list is assumed to contain values for periods $1, \ldots, T$ in elements $0, \ldots, T-1$.
* As a list with $T+1$ elements, in which case the list is assumed to contain values for periods $1, \ldots, T$ in elements $1, \ldots, T$, and the 0th element is ignored.

For example, if a given problem instance has 3 time periods and a parameter `holding_cost` equals 1 in every time period, the following are equivalent:
```python
holding_cost=1
holding_cost=[1, 1, 1]
holding_cost=[0, 1, 1, 1]
```
Similarly, if a parameter `demand` equals 10, 30, and 25 in periods 1--3, respectively, then the following are equivalent:
```python
demand=[10, 30, 25]
demand=[0, 10, 30, 25]
```
In most or all functions, you may mix input types, specifying some as singletons and some as lists.
Time-based output parameters returned by Stockpyl functions are always given as lists with $T+1$ elements.

## 2.6 The Wagner–Whitin Problem

The Wagner–Whitin problem (sometimes referred to as the dynamic economic lot-sizing (DEL) or uncapacitated lot-sizing (ULS) model) is a periodic-review, finite-horizon, deterministic inventory model that makes decisions about which periods to order in, and how much to order in each of those periods. Like the EOQ model, it assumes deterministic demand, but unlike the EOQ, it is a periodic-review model and it allows the demand to vary from one period to another.

The `wagner_whitin` module solves the problem using the classic dynamic programming (DP) algorithm by Wagner and Whitin (1958). The module contains a single function, [`wagner_whitin()`](https://stockpyl.readthedocs.io/en/latest/api/seio/wagner_whitin.html#stockpyl.wagner_whitin.wagner_whitin), which implements the Wagner–Whitin DP recursion (see _FoSCT_ §3.7.3). The function has the signature
```python
wagner_whitin(num_periods, holding_cost, fixed_cost, demand, purchase_cost=0)
```
and returns four parameters: `order_quantities` ($Q$), `cost` ($\theta_1$), `costs_to_go` ($\theta$), and `next_order_periods` ($s$). The optional input parameter `purchase_cost` allows you to specify a purchase cost in each time period.
With the exception of `num_periods`, the input parameters may vary by time period; see [Section 2.5](#sec_2_5). The output arrays are 1-indexed; i.e., they have length $T+1$, where element $t$ gives the value for period $t$, and element 0 is ignored.

**Example:** In _FoSCT_ Example 3.9, there are 4 periods; the holding cost is 2, the fixed cost is 500, and the demands in periods 1, ..., 4 are 90, 120, 80, and 70, respectively. Solving this problem, we get an optimal solution in which we order in periods 1 and 3, with order quantities 210 and 150 and a total cost of 1380:

In [ ]:
from stockpyl.wagner_whitin import wagner_whitin
Q, cost, theta, s = wagner_whitin(4, 2, 500, [90, 120, 80, 70])

In [ ]:
# Display order quantity list and cost.
Q, cost

In [ ]:
# Display costs-to-go and next order periods.
theta, s

Note that we could alternatively specify the holding and/or fixed costs as lists in which every element is the same, e.g.:

In [ ]:
Q, cost, theta, s = wagner_whitin(4, [2, 2, 2, 2], 500, [90, 120, 80, 70])
Q, cost

---
**Exercise:** Suppose there are 6 time periods. The holding cost per item per period is 0.1, the fixed cost is 7, and the demands in periods 1, ..., 6 are 12, 16, 17, 0, 9, and 13, respectively. Solve the Wagner–Whitin problem.

## 2.7 Stochastic Finite-Horizon Problems

The `finite_horizon` module contains code for solving finite-horizon, stochastic inventory optimization problems, with or without fixed costs, under normally distributed demands, using DP. (See _FoSCT_ §4.3.3 and §4.4.3.)

The main function in the module is [`finite_horizon_dp()`](https://stockpyl.readthedocs.io/en/latest/api/seio/finite_horizon.html#stockpyl.finite_horizon.finite_horizon_dp), which has the signature
```python
finite_horizon_dp(num_periods, holding_cost, stockout_cost,
    terminal_holding_cost, terminal_stockout_cost, purchase_cost, fixed_cost,
    demand_mean, demand_sd, discount_factor=1.0, initial_inventory_level=0.0,
    trunc_tol=0.02, d_spread=4, s_spread=5)
```
and returns six parameters: `reorder_points` ($s$), `order_up_to_levels` ($S$), `total_cost` ($\theta_1(x_1)$), `cost_matrix` ($\theta_t(x)$), `oul_matrix` ($y_t(x)$), and `x_range`.
Most of the input parameters may vary by time period; see [Section 2.5](#sec_2_5). The output arrays are 1-indexed; i.e., they have length $T+1$, where element $t$ gives the value for period $t$, and element 0 is ignored.

The function assumes that there is a termal cost function of the form
$$\theta_{T+1}(x) = h_{T+1}x^+ + p_{T+1}x^-,$$
where $h_{T+1}$ and $p_{T+1}$ are the terminal holding and stockout costs, respectively, and
	\begin{align*}
		x^+ & \equiv \max\{x,0\} \\
		x^- & \equiv \max\{-x,0\}.
	\end{align*}

If $K=0$ in each time period, then a base-stock policy is optimal and the output parameters will reflect that; that is, the order-up-to level in period $t$ (as a function of the period-$t$ starting inventory $x$) will have the form
	$$ y_t(x) = \begin{cases} S_t, & \text{if $x \le S_t$} \\ 0, & \text{if $x > S_t$,} \end{cases} $$
and `reorder_points[t]` will equal `order_up_to_levels[t]` for all periods `t`. If $K \ne 0$, then an $(s,S)$ policy is optimal and the output parameters will reflect that; that is, the order-up-to levels will have the form
	$$ y_t(x) = \begin{cases} S_t, & \text{if $x \le s_t$} \\ 0, & \text{if $x > s_t$.} \end{cases} $$

**Example:** Suppose we have $T=5$ time periods; in each time period the holding cost is 1, the stockout cost is 20, the purchase cost is 2, the fixed cost is 50, the demand is distributed as $N(100,20^2)$; and the terminal costs are $h_6 = 1$ and $p_6 = 20$. Solving this problem gives the following $(s,S)$ pairs, with expected cost 1558.7 over the time horizon:
\begin{align*}
(s_1,S_1) & = (110, 133) \\
(s_2,S_2) & = (110, 133) \\
(s_3,S_3) & = (110, 133) \\
(s_4,S_4) & = (110, 133) \\
(s_5,S_5) & = (111, 126)
\end{align*}

In [ ]:
from stockpyl.finite_horizon import finite_horizon_dp
T = 5
h = 1
p = 20
h_term = 1
p_term = 20
c = 2
K = 50
mu = 100
sigma = 20
s, S, cost, _, _, _ = finite_horizon_dp(T, h, p, h_term, p_term, c, K, mu, sigma)
s, S

Let's confirm that if $K=0$, the $s$ values equal the $S$ values, i.e., we have a base-stock policy:

In [ ]:
K = 0
s, S, cost, _, _, _ = finite_horizon_dp(T, h, p, h_term, p_term, c, K, mu, sigma)
s, S

The input parameters `trunc_tol`, `d_spread`, and `s_spread`, and the output parameter `x_range`, relate to the truncation and discretization performed within the function. For more details, see the [online documentation](https://stockpyl.readthedocs.io/en/latest/api/seio/finite_horizon.html#stockpyl.finite_horizon.finite_horizon_dp) or the TutORial.

---
**Exercise:** Suppose we have $T=4$ time periods. In each time period the holding cost is 5, the stockout cost is 120, and the fixed cost is 50.  The purchase cost in periods 1,..., 4 is 10, 40, 15, and 15, respectively. The demand mean in periods 1, ..., 4 is 150, 360, 210, and 90, respectively, and the standard deviation is 10 in every period. The terminal holding and stockout costs are 10 and 250, respectively. Find the optimal $s$ and $S$ values.

**Next Up:** Stockpyl Tutorial §3: Supply Uncertainty